In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install mne

In [ ]:

import os
import numpy as np
import mne
from mne.channels import compute_native_head_t, read_custom_montage
from mne.viz import plot_alignment
from scipy import signal

# sample_data_folder = mne.datasets.sample.data_path()

In [ ]:
# fine_cal_file = os.path.join(sample_data_folder, 'SSS', 'sss_cal_mgh.dat')
# crosstalk_file = os.path.join(sample_data_folder, 'SSS', 'ct_sparse_mgh.fif')

In [ ]:
# DATA LOAD

In [ ]:
path = '/content/drive/My Drive/coding/data/EEGdata/data'
file_list = os.listdir(path)
data_list = file_list[1:]
# data_list

In [ ]:
raw = mne.io.read_raw_edf("/content/drive/My Drive/coding/data/EEGdata/data/p5_lev1_no_M_raw.edf",preload=True)
loc = mne.channels.read_custom_montage("/content/drive/My Drive/coding/data/EEGdata/data/DSI_channelLocs.locs") 
mne.set_log_level("WARNING")

In [ ]:
loc.plot() # checking the loc we use

In [ ]:
# loc.ch_names

In [ ]:
print(raw)
print(raw.info)

In [ ]:
raw.info['bads']

In [ ]:
channels = raw.ch_names
print(channels[0:19])
change_ch = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']
print(loc.ch_names)

In [ ]:
# data = .get_data()

In [ ]:
chns = raw.info["ch_names"][0:19]
print(chns)

raw1 = raw.copy().pick_channels(chns) # exclude the stimulus chn
raw1.info

In [ ]:
    for i,k in zip(chns, change_ch) :
        mne.channels.rename_channels(raw1.info, {str(i):str(k)} ) # change the ch names to match the loc information

In [ ]:
raw1.info

In [ ]:
print(raw1.info.ch_names)
print(loc.ch_names)

In [ ]:
raw1.set_montage(loc)
raw1.info
raw1.info['dig'] # check fot loc info injected 

In [ ]:
pick_feats = ['Fp1 - Vref', 'Fp2 - Vref', 'F7 - Vref', 'F3 - Vref', 
         'Fz - Vref', 'F4 - Vref', 'F8 - Vref', 'T3 - Vref', 
         'C3 - Vref', 'Cz - Vref', 'C4 - Vref', 'T4 - Vref', 
         'P3 - Vref', 'Pz - Vref', 'P4 - Vref'] 
         
# exclude bads chans though plotting in below process

In [ ]:
# raw1.plot(lowpass=20, n_channels=21, highpass=1 ,duration=100)

In [ ]:
# bads = ['P7', 'P8', 'O1', 'O2'] # Can identify bad(noisy) channels

In [ ]:
# # bads = ['P7', 'P8', 'O1', 'O2'] # Can identify bad(noisy) channels
# raw1.info['bads']=bads
# raw1.info

In [ ]:
# raw1.plot(block=True, lowpass=100, n_channels=21 ,scalings='auto') 
# Can check the bad chns append or not

In [ ]:
# picks = mne.pick_types(raw1.info, exclude='bads')

In [ ]:
raw1.info

In [ ]:
raw1.plot_psd()

In [ ]:
# pick_raw.plot_psd()

In [ ]:
# raw.info['bads']

In [ ]:
# Line Noise Removal (Notch Filter)

In [ ]:
np.arange(60,185,60)

In [ ]:
raw_nf = raw1.copy().notch_filter(np.arange(60,180,60)) # Notch filtering to remove 60 hz and its multiples

In [ ]:
# pick_raw_nf = pick_raw.copy().notch_filter(np.arange(60,180,60)) # Notch filtering to remove 60 hz and its multiples

In [ ]:
# raw_nf.plot(n_channels=21, highpass=1, lowpass=10 ,duration=120,remove_dc=True)

In [ ]:
# pick_raw_nf.plot(block=True, lowpass=13 ,scalings='auto')

In [ ]:
raw_nf.plot_psd()

In [ ]:
# pick_raw_nf.plot_psd()

In [ ]:
# Band Pass Filter

In [ ]:
raw_bp = raw_nf.copy().filter(1, 100, n_jobs=1) # Bandpass filter 1 to 100 hz
raw_bp.info

In [ ]:
# pick_raw_bp = pick_raw_nf.copy().filter(1,100, n_jobs=1) # Bandpass filter 1 to 100 hz
# pick_raw_bp.info

In [ ]:
# raw1.info
# raw_nf.info
# raw_bp.info

In [ ]:
# pick_raw1.info
# pick_raw_nf.info
# pick_raw_bp.info

In [ ]:
raw_bp.plot(duration=60, highpass=4, lowpass=13, start=40)

In [ ]:
# pick_raw_bp.plot(block=True, lowpass=13, n_channels=21 ,scalings='auto')

In [ ]:
raw_bp.plot_psd()

In [ ]:
# pick_raw_bp.plot_psd()

In [ ]:
## Bad Channel Rejection & Interpolate

In [ ]:
bads = [ 'Fp1','Fp2', 'O1', 'O2'] # Can identify bad(noisy) channels
raw_bcr = raw_bp.copy()
raw_bcr.info['bads']=bads
print(raw_bcr.info)

raw_bcr.plot(duration=60, highpass=4, lowpass=13, start=40)

In [ ]:
raw_intp = raw_bcr.copy()
raw_intp = raw_intp.interpolate_bads(reset_bads=False) # interpolate bad chns

In [ ]:
eeg_data = raw1.copy()
eeg_data_interp = raw_intp.copy()

for title, data in zip(['orig.', 'interp.'], [eeg_data, eeg_data_interp]):
    fig = data.plot(butterfly=True, color='#00000022',duration=60, start=40, lowpass=13, highpass=4)
    fig.subplots_adjust(top=1)
    fig.suptitle(title, size='xx-large', weight='bold')

In [ ]:
raw_intp.plot(duration=60, start=40, highpass=4, lowpass=13)

In [ ]:
raw_intp.info

In [ ]:
## Re-Referencing

In [ ]:
raw_rf = raw_intp.copy()
raw_rf = raw_rf.set_eeg_reference(ref_channels='average')

In [ ]:
raw_rf.plot(duration=60, start=40, highpass=4, lowpass=13)

In [ ]:
eeg_data1 = raw1.copy()
eeg_data_interp1 = raw_rf.copy()

for title, data in zip(['orig.', 'interp.'], [eeg_data1, eeg_data_interp1]):
    fig = data.plot(butterfly=True, color='#00000022', duration=60, start=40, highpass=4, lowpass=13)
    fig.subplots_adjust(top=1)
    fig.suptitle(title, size='xx-large', weight='bold')

In [ ]:
## Artefact removal

In [ ]:
raw_rf.plot(duration=60, start=40, n_channels=19, remove_dc=False, highpass=30, lowpass=100)

In [ ]:
# regexp = ['Fz','F3', 'F4', 'Cz', 'C3', 'C4' 'T7', 'T8', 'P7', 'P8', 'Pz', 'P3', 'P4']
# artifact_picks = mne.pick_channels_regexp(raw_rf.ch_names, regexp)

In [ ]:
filt_low = raw_rf.copy()
filt_low.load_data().filter(l_freq=1., h_freq=None)

In [ ]:
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)

ica = ICA(n_components=13, random_state=97)
ica.fit(filt_low)

In [ ]:
raw.load_data()
ica.plot_sources(raw_rf)

In [ ]:
ica.plot_components()

In [ ]:
ica.plot_properties(raw_rf, picks=[0,1,2,3,4,5,6,7,8,9,10,11,12])

In [ ]:
reconst_raw = raw_rf.copy()
reconst_raw = ica.apply(reconst_raw)

reconst_raw.plot(duration=60, start=40 , highpass=4, lowpass=13 )

In [ ]:
reconst_raw.info

In [ ]:
## Filter from 4 to 13 

In [ ]:
fin_data = reconst_raw.copy()
fin_data.filter(4,13) # we are going to see the alpha and theta which have range 4 to 13

In [ ]:
fin_data.info

In [ ]:
fin_data.plot(start= 40, duration=60, lowpass=4, highpass=13)

In [ ]:
fin_data.plot_psd()

In [ ]:
fin_data.plot_sensors(kind='topomap',)

In [ ]:
p1_lev1_no = fin_data.get_data()

In [ ]:
p1_lev1_no.shape

In [ ]:
list_ex = []

In [ ]:
c = p1_lev1_no[3].tolist()
list_ex.append(c)

In [ ]:
list_ex